<a href="https://colab.research.google.com/github/mrpintime/Constraints_NeuralNet/blob/main/Constraints_NeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 7  


#Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Generative AI/Excercises/SeatArrangement

/content/drive/MyDrive/Generative AI/Excercises/SeatArrangement


# Approche 1  
Several Conflict Matrix

In [120]:
import random
import numpy as np

list_of_conflicts = []

while len(list_of_conflicts) < 50:
    pairs_list = set()
    matrix = np.zeros((24, 24), dtype=int)

    while matrix.sum() < 40:
        num1 = np.random.choice(range(24))
        num2 = np.random.choice(range(24))

        if num1 == num2:
            continue

        pair = (num1, num2)
        if pair in pairs_list:
            continue

        pairs_list.add(pair)
        matrix[num1, num2] = 1

    if not any(np.array_equal(matrix, conflict) for conflict in list_of_conflicts):
        list_of_conflicts.append(matrix)

In [121]:
conflicts = np.array(list_of_conflicts)

In [5]:
conflicts.shape

(570, 24, 24)

In [175]:
def create_adjacent_matrix(n_seats, seats_per_row, seats_per_col):

    adjacent_mask = np.zeros((n_seats, n_seats))

    for i in range(n_seats):
        if i % seats_per_row != 0:
            adjacent_mask[i, i-1] = 1
        if i % seats_per_row != seats_per_row-1:
            adjacent_mask[i, i+1] = 1
        if i >= seats_per_row:
            adjacent_mask[i, i-seats_per_row] = 1
        if i < n_seats-seats_per_row:
            adjacent_mask[i, i+seats_per_row] = 1

    return adjacent_mask

adjacent_mask = create_adjacent_matrix(24,6,4)

## Neural network and training

In [9]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as op

In [11]:
class SeatingArr(nn.Module):
    def __init__(self):
        super(SeatingArr, self).__init__()
        self.flatten = nn.Flatten()
        # self.embedding = nn.Embedding(24, 10)  # Embedding layer for 24 people
        self.fc1 = nn.Linear(24*24, 128, dtype=torch.float64)         # First fully connected layer
        self.fc2 = nn.Linear(128, 64, dtype=torch.float64)            # Second fully connected layer
        self.fc3 = nn.Linear(64, 24*24, dtype=torch.float64)          # Output layer

    def forward(self, x):
        x = self.flatten(x)
        # embedded = self.embedding(x)           # Embed each person
        # x = embedded.view(-1, 240)             # Flatten into a vector
        x = F.relu(self.fc1(x))                # Apply first fully connected layer
        x = F.relu(self.fc2(x))                # Apply second fully connected layer
        x = F.relu(self.fc3(x))                        # Output layer (no activation)
        x = x.view(-1, 24, 24)                 # Reshape to 24x24 matrix
        x = F.softmax(x, dim=2)
        return x


In [122]:
torch_conflicts = torch.tensor(conflicts, dtype=torch.float64)
loader = torch.utils.data.DataLoader(torch_conflicts, batch_size=64, pin_memory=True)
examples = enumerate(loader)
batch_idx, example_data = next(examples)

In [14]:
batch_idx, example_data.shape, loader.batch_size, loader.dataset.shape, len(loader.dataset), len(loader)

(0, torch.Size([64, 24, 24]), 64, torch.Size([570, 24, 24]), 570, 9)

In [15]:
example_data.type()

'torch.DoubleTensor'

In [110]:
adjacent_mask = create_adjacent_matrix(n_seats=24, seats_per_row=6, seats_per_col=4)
adjacent_mask_torch = torch.tensor(adjacent_mask, dtype=torch.float64)

def calculate_conflict_torch(seating_arrangement, conflict_matrix):
    ca_mul = conflict_matrix * adjacent_mask_torch
    conflicts = torch.sum(torch.matmul(seating_arrangement.type(torch.float64), ca_mul))
    return conflicts

def custom_loss_torch(predicted_seating_arrangement, conflicts_tensor):
    high_factor = 10
    loss = 0
    batch_size = predicted_seating_arrangement.shape[0]

    # Ensure the predicted seating arrangement is in float64
    predicted_seating_arrangement = predicted_seating_arrangement.type(torch.float64)

    # Calculate Conflict in produced seating arrangement
    conflict = calculate_conflict_torch(predicted_seating_arrangement, conflicts_tensor)
    # Ensure each seat is assigned to only one person (columns should sum to 1) (Uniqueness)
    col_sum_loss = torch.sum((torch.sum(predicted_seating_arrangement, dim=1) - 1) ** 2) * high_factor
    # Ensure each person gets a unique seat (rows should sum to 1) (Uniqueness)
    row_sum_loss = torch.sum((torch.sum(predicted_seating_arrangement, dim=2) - 1) ** 2) * high_factor

    loss = row_sum_loss + col_sum_loss + conflict

    return loss / batch_size # Normalize

In [111]:
# save model every 10 epochs
interval = 20

def Training(epoch):
    # train
    net.train()
    for batch_idx, conflict in enumerate(loader):
        optim.zero_grad()
        output = net(conflict)
        loss = custom_loss_torch(output, conflict)
        loss.backward()
        optim.step()
    if epoch % interval == 0:
        print('Train Epoch: {} Loss: {:.6f}'.format(epoch, loss.item()))
        # save state of model and optimizer
        torch.save(net.state_dict(), './model.pth')
        torch.save(optim.state_dict(), './optimizer.pth')


In [177]:
net = SeatingArr()
optim = op.Adam(params=net.parameters(), lr=0.00001)

In [178]:
n_epoch = 3000
for i in range(n_epoch):
     Training(i+1)

Train Epoch: 20 Loss: 5.777636
Train Epoch: 40 Loss: 5.774256
Train Epoch: 60 Loss: 5.771038
Train Epoch: 80 Loss: 5.767974
Train Epoch: 100 Loss: 5.764996
Train Epoch: 120 Loss: 5.762107
Train Epoch: 140 Loss: 5.759288
Train Epoch: 160 Loss: 5.756503
Train Epoch: 180 Loss: 5.753747
Train Epoch: 200 Loss: 5.751007
Train Epoch: 220 Loss: 5.748274
Train Epoch: 240 Loss: 5.745515
Train Epoch: 260 Loss: 5.742765
Train Epoch: 280 Loss: 5.740006
Train Epoch: 300 Loss: 5.737224
Train Epoch: 320 Loss: 5.734447
Train Epoch: 340 Loss: 5.731685
Train Epoch: 360 Loss: 5.728987
Train Epoch: 380 Loss: 5.726322
Train Epoch: 400 Loss: 5.723731
Train Epoch: 420 Loss: 5.721234
Train Epoch: 440 Loss: 5.718772
Train Epoch: 460 Loss: 5.716318
Train Epoch: 480 Loss: 5.713906
Train Epoch: 500 Loss: 5.711464
Train Epoch: 520 Loss: 5.709019
Train Epoch: 540 Loss: 5.706546
Train Epoch: 560 Loss: 5.704107
Train Epoch: 580 Loss: 5.701725
Train Epoch: 600 Loss: 5.699360
Train Epoch: 620 Loss: 5.697032
Train Epoch:

In [179]:
with torch.no_grad():
    output = net(torch_conflicts)
    loss = custom_loss_torch(output, torch_conflicts)
    loss_val = loss.item()
    model_out_readable = torch.argmax(output, dim=2)

In [180]:
output[0]

tensor([[0.0481, 0.0388, 0.0388, 0.0484, 0.0474, 0.0413, 0.0388, 0.0388, 0.0388,
         0.0404, 0.0501, 0.0396, 0.0388, 0.0469, 0.0441, 0.0388, 0.0442, 0.0438,
         0.0388, 0.0388, 0.0388, 0.0388, 0.0388, 0.0402],
        [0.0374, 0.0374, 0.0379, 0.0374, 0.0374, 0.0414, 0.0440, 0.0536, 0.0376,
         0.0374, 0.0401, 0.0485, 0.0374, 0.0447, 0.0430, 0.0607, 0.0374, 0.0409,
         0.0374, 0.0449, 0.0374, 0.0461, 0.0430, 0.0374],
        [0.0466, 0.0508, 0.0373, 0.0384, 0.0377, 0.0373, 0.0418, 0.0373, 0.0424,
         0.0573, 0.0383, 0.0381, 0.0373, 0.0373, 0.0373, 0.0387, 0.0391, 0.0437,
         0.0373, 0.0493, 0.0386, 0.0487, 0.0429, 0.0469],
        [0.0434, 0.0386, 0.0496, 0.0493, 0.0386, 0.0386, 0.0395, 0.0386, 0.0386,
         0.0443, 0.0395, 0.0432, 0.0386, 0.0386, 0.0453, 0.0441, 0.0386, 0.0447,
         0.0386, 0.0431, 0.0496, 0.0386, 0.0386, 0.0396],
        [0.0519, 0.0377, 0.0447, 0.0526, 0.0377, 0.0377, 0.0442, 0.0401, 0.0443,
         0.0377, 0.0408, 0.0426, 0.0391

In [181]:
s = 0
for i in range(24):
  s+=output[0][4][i]
print(s)

tensor(1.0000, dtype=torch.float64)


In [182]:
model_out_readable[0]

tensor([10, 15,  9, 20,  3, 21, 14, 15, 16, 13,  7, 21,  7,  3, 12, 20, 12, 15,
         1,  9,  7, 12,  2,  5])

In [183]:
model_out_readable[2].view(4,6)

tensor([[17, 11,  6,  2, 22,  1],
        [ 4,  4, 12,  3, 11, 21],
        [ 7, 13, 18,  1, 10, 15],
        [ 1, 13, 21, 13, 14,  5]])

In [188]:
def repair_output(output):
    remain_persons = []
    for j in range(len(output)):
        if j not in output:
          remain_persons.append(j)
    if len(remain_persons) == 0:
        return output
    for i in range(1, len(output)-1):
        if output[i] in output[i+1:] or output[i] in output[:i-1]:
            output[i] = remain_persons.pop()

    return output

new_output = repair_output(model_out_readable[20])


In [189]:
print(new_output.view(4,6))

tensor([[16,  8, 23, 22, 19, 14],
        [ 4, 13,  3, 12, 11, 10],
        [ 9,  5, 18, 20,  7, 15],
        [ 1,  0, 21, 17,  6,  2]])


In [190]:
new_output = new_output.view(4,6).numpy()
for i in range(4):
  for j in range(5):
    if conflicts[0][new_output[i][j]][new_output[i][j+1]] == 1:
      print('person in row', i, 'and col', j, 'has conflict with', new_output[i][j+1])

person in row 0 and col 3 has conflict with 19
person in row 2 and col 4 has conflict with 15


In [ ]:
# Entropy Loss Function

In [141]:
print(conflicts[0][20][17])

0


In [131]:
a.pop()

4